In [ ]:
!pip install sentence-transformers

In [ ]:
import spacy 
nlp = spacy.load('en_core_web_sm') 

In [ ]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

In [ ]:
import json
import os
from sentence_transformers import SentenceTransformer, util



In [ ]:
df = pd.read_csv('ccc-organizations-2011_1.csv')

papers = df.Description.values

print(len(papers), "Charities")

entity =[]
def get_entity(text):
    name_org = []
    doc = nlp(text) 
    for ent in doc.ents:
        name_org.append(str(ent.text) + " " + str(ent.label_))
    return name_org

for i in range(len(papers)):
    entity.append(get_entity(papers[i]))

print(entity)
#We then load the allenai-specter model with SentenceTransformers
model = SentenceTransformer('allenai-specter')

#To encode the descriptions to a single string
paper_texts = [paper for paper in papers]

#Compute embeddings for all descriptions
corpus_embeddings = model.encode(paper_texts, convert_to_tensor=True)


318 Charities
[[], ['Maryland GPE'], [], [], ['Gwynns Falls Elementary School ORG'], ['Justice Policy Institute ORG'], [], [], ['RIF ORG'], ['Paul Newman PERSON'], ['ALS ORG', "Lou Gehrig's PERSON"], ['Baltimore City GPE'], [], [], ['The International Rescue Committee ORG'], [], [], [], ['the Chesapeake Bay LOC'], ['Assists NORP', 'Maryland GPE'], [], ['Christmas DATE'], ['Maryland GPE'], [], [], [], [], ['five CARDINAL', 'Carroll County GPE'], [], [], ['Baltimore GPE', 'CCC ORG', '100% PERCENT'], ['American NORP'], ['1864 DATE'], ['Maryland GPE'], [], [], ['Maryland GPE'], ['Maryland GPE'], ['Anne Arundel County GPE'], ['Annual DATE', 'Give a Child a Christmas WORK_OF_ART'], ['KIPP Baltimore PERSON', 'Baltimore City GPE', 'four year DATE'], ['N-12 CARDINAL'], ['Kennedy Krieger institute ORG'], [], [], ['Jubilee Baltimore PERSON', 'Baltimore GPE', 'Jubilee PERSON', 'Baltimore GPE'], ['One CARDINAL', 'America GPE', 'the United States GPE', '1979 DATE'], [], [], ['Baltimore GPE'], [], ['

In [18]:
res = [ele for ele in entity if ele != []]

def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        # else:
        #     flat_list.append(element)
    return flat_list

all_entities = flatten_list(res)

In [ ]:
all_entities

In [ ]:

substring = "GPE"

named_org = [string for string in all_entities if substring in string]

named_org

In [24]:
from collections import Counter

Counter = Counter(named_org)
most_occuring = Counter.most_common()
  
print(most_occur)

[('Baltimore GPE', 27), ('Maryland GPE', 26), ('Baltimore City GPE', 20), ('Catholic NORP', 6), ('America GPE', 5), ('Assists NORP', 4), ('one CARDINAL', 4), ('Jewish NORP', 4), ('American NORP', 3), ('Anne Arundel County GPE', 3)]


In [ ]:
from termcolor import colored

charities = []
def search_papers(title):
   query_embedding = model.encode(title+'[SEP]', convert_to_tensor=True) # Converts to tensor
   search_hits = util.semantic_search(query_embedding, corpus_embeddings)
   search_hits = search_hits[0]
   count = 0
   top_header = "Top 10 charities related"
   print(colored(top_header,'blue'))
   for hit in search_hits:
         related_paper = papers[hit['corpus_id']]
         count += 1
         print(str(count)+ ")" + " Description of charity : " +related_paper)
         print("similiarity score of " + str(format(search_hits[count-1]['score'],".2f")))
         subsetDataFrame = df[df['Description'] == related_paper]
         k=subsetDataFrame.values
         a_string = "Name of Organization"
         print("\033[1m" + a_string + "\033[0m")
         print(colored(str(k[0][1]), 'red'))
         print(" located in " + str(k[0][7] + " , Telephone : " + str(k[0][3])))
         print( "Street address: " + str(k[0][9])+ "\n")
         
#search_papers(title=input(str()))

search_papers("eyes")

Top 10 charities related
1) Description of charity : Provides human eye tissue for sight restoring corneal transplant surgery. Directly supports research into causes of blindness.
similiarity score of 0.73
Name of Organization
Tissue Banks International
 located in Baltimore , Telephone : 410-783-0183
Street address: 815 Park Avenue
Baltimore, MD 21201
(39.29897226908366, -76.61848419245842)

2) Description of charity : Enables 30,000 Marylanders annually to avoid needless blindness from eye disease and injury through eye screenings, eye examinations and consumer eye health and safety information.
similiarity score of 0.72
Name of Organization
Maryland Society for Sight, Inc.
 located in Baltimore , Telephone : 410-889-2505
Street address: 1313 Old Cold Spring Lane
Baltimore, MD 21209
(39.34349770275429, -76.64182159911877)

3) Description of charity : Researches to discover treatments and cures for retinal diseases that cause blindness such as retinitis pigmentosa, macular degeneratio